# Examen final Introducción a la Inteligencia artificial

## 14/08/2020

## Referencias

- [Cheatsheet de Latex](https://www.caam.rice.edu/~heinken/latex/symbols.pdf)
- [DRIVE](https://drive.google.com/drive/folders/1TDRvlwfSgLI39gHXgVhWlSaRJ-aTSaDE?usp=sharing)

## Comentarios sobre el estado del documento



## Importar librerías

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# import seaborn as sns

np.random.seed(42)

#PANDAS
# import pandas as pd

%matplotlib inline

In [ ]:
sys.path.insert(1, 'C:/Users/jota_/00_Especialización_IA/Examenes')

from IIA_preprocessing import Data
from IIA_models import
from IIA_evaluation import

X = Data
filepath = 'C:/Users/jota_/00_Especialización_IA/00_Recursos/01_DataSets/income.data.csv'
Structure = np.dtype([('index', np.float32), ('X', np.float32), ('y', np.float32)])
data = X(filepath, Structure)